# Cel biznesowy

Celem klasteryzacji wybranego zbioru danych finansowych spółek notowanych na Borsa İstanbul (BIST) jest **grupowanie spółek o podobnych profilach finansowych na podstawie ich bilansów rocznych**.

Dzięki segmentacji spółek według cech takich jak aktywa, pasywa, zyski czy wskaźniki finansowe, możliwe będzie:

- **Identyfikowanie wzorców finansowych** charakterystycznych dla różnych sektorów i etapów rozwoju spółek,
- **Wsparcie decyzji inwestycyjnych i analizy ryzyka** poprzez rozróżnienie grup spółek o stabilnych, rosnących lub problematycznych fundamentach finansowych,
- **Ułatwienie analizy porównawczej** i benchmarkingu wśród firm o podobnej wielkości lub profilu działalności,
- **Automatyzacja analizy dużych zbiorów danych finansowych**, które pochodzą z różnorodnych, nieustrukturyzowanych źródeł, w celu ich lepszego wykorzystania w modelach predykcyjnych i badaniach naukowych.

# Etap 1.
## Wczytanie surowych danych według roku, tłumaczenie nazw kolumn
## zapis do pliku "Data/Concatenated Features/raw_features.csv"

In [1]:
# run only once
#!pip install deep-translator

In [2]:
import pandas as pd
import numpy as np
import os
from glob import glob
from deep_translator import GoogleTranslator

In [3]:
# Ścieżka do folderu z plikami 
folder_path = "Data/Turkish Public Companies Balance Sheets from KAP"

# Lista wszystkich plików .xlsx 
excel_files = glob(os.path.join(folder_path, "*.xlsx"))

# Lista do zebrania przetworzonych DataFrame'ów
df_list = []

for file in excel_files:
    # Wczytaj plik
    df = pd.read_excel(file)
    
    # Pivot: z długiego na szeroki format (zmiana układu danych)
    df_wide = df.pivot_table(index=[], columns='Desc', values='Yıllık', aggfunc='first').reset_index()
    
    # Dodaj kolumnę z nazwą pliku (bez ścieżki)
    df_wide['source_file'] = os.path.basename(file)
    
    # Dodaj do listy
    df_list.append(df_wide)

# Połącz wszystkie w jeden DataFrame
df = pd.concat(df_list, ignore_index=True)

# Dodanie kolumn identyfikujących spółkę i rok na podstawie nazwy pliku
df['Nazwa'] = df['source_file'].str.extract(r'^([^_]+)')[0]
df['Rok'] = df['source_file'].str.extract(r'_([0-9]{4})')[0]


In [4]:
# Tłumaczymy wszystkie nazwy kolumn z tureckiego na polski
translated_columns = [GoogleTranslator(source='tr', target='pl').translate(col) for col in df.columns]
df.columns = translated_columns

In [5]:
print("Przetłumaczone kolumny:")
len( translated_columns)
df.head()

Przetłumaczone kolumny:


,indeks,(SubTotal),Przepisy długoterminowe,Amortyzacja i straż pożarna,Kapitał partnerstwa głównego,Akcje mniejszościowe,Przepisy zadłużenia,Żywe istoty,Zachęta państwowa i pomoc,Zwiększenie wartości fundusze,...,Zobowiązania długoterminowe,Inne dochody i wydatki przed opodatkowaniem,Yab. Money Trans. Różnica. Efekt. Poprzedni.nak.ve nak. Tak jak. Wzrost/spadek netto,Sprzedaż zagraniczna,Sprzedaż krajowa,Akcje zysków/strat inwestycji cenianych metodą samowystarczalności,Samo -właściwość,Źródło_pliku,Nazwa,Rok
0,Yıllık,4.524640e+08,678404.0,1242248.0,1.387926e+08,0.0,2852756.0,0.0,0.0,0.0,...,2806817.0,449296.0,-9990337.0,0.0,0.0,0.0,1.387926e+08,A1CAP_2020.xlsx,A1CAP,2020
1,Yıllık,8.515088e+08,0.0,3263754.0,2.966258e+08,0.0,6670834.0,0.0,0.0,0.0,...,8695590.0,777230.0,121976765.0,0.0,0.0,0.0,2.966258e+08,A1CAP_2021.xlsx,A1CAP,2021
2,Yıllık,4.019261e+09,4748296.0,12651941.0,1.487849e+09,0.0,70752251.0,0.0,0.0,0.0,...,132279622.0,17107575.0,463136619.0,0.0,0.0,0.0,1.487849e+09,A1CAP_2022.xlsx,A1CAP,2022
3,Yıllık,6.297396e+09,7715324.0,21452328.0,3.696944e+09,0.0,42331251.0,0.0,0.0,0.0,...,34025698.0,6055765.0,691863980.0,0.0,0.0,0.0,3.696944e+09,A1CAP_2023.xlsx,A1CAP,2023
4,Yıllık,8.297508e+09,9773361.0,31826374.0,4.065838e+09,0.0,28062814.0,0.0,0.0,0.0,...,160295553.0,5169826.0,369461172.0,0.0,0.0,0.0,4.065838e+09,A1CAP_2024.xlsx,A1CAP,2024


In [13]:
# Zapis formatu długiego do pliku CSV 
output_path = "Data/Concatenated Features/by_year_features_long.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [16]:
df= pd.read_csv("Data/Concatenated Features/by_year_features_long.csv")
#df= df.fillna(0)
df.head()

,(SubTotal),Przepisy długoterminowe,Amortyzacja i straż pożarna,Kapitał partnerstwa głównego,Akcje mniejszościowe,Przepisy zadłużenia,Żywe istoty,Zachęta państwowa i pomoc,Zwiększenie wartości fundusze,Inne należności,...,Inne zyski z działalności handlowej (szkody),Zobowiązania długoterminowe,Inne dochody i wydatki przed opodatkowaniem,Yab. Money Trans. Różnica. Efekt. Poprzedni.nak.ve nak. Tak jak. Wzrost/spadek netto,Sprzedaż zagraniczna,Sprzedaż krajowa,Akcje zysków/strat inwestycji cenianych metodą samowystarczalności,Samo -właściwość,Nazwa,Rok
0,4.524640e+08,678404.0,1242248.0,1.387926e+08,0.0,2852756.0,0.0,0.0,0.0,25621371.0,...,0.0,2806817.0,449296.0,-9990337.0,0.0,0.0,0.0,1.387926e+08,A1CAP,2020
1,8.515088e+08,0.0,3263754.0,2.966258e+08,0.0,6670834.0,0.0,0.0,0.0,3489822.0,...,0.0,8695590.0,777230.0,121976765.0,0.0,0.0,0.0,2.966258e+08,A1CAP,2021
2,4.019261e+09,4748296.0,12651941.0,1.487849e+09,0.0,70752251.0,0.0,0.0,0.0,24810952.0,...,0.0,132279622.0,17107575.0,463136619.0,0.0,0.0,0.0,1.487849e+09,A1CAP,2022
3,6.297396e+09,7715324.0,21452328.0,3.696944e+09,0.0,42331251.0,0.0,0.0,0.0,40534909.0,...,0.0,34025698.0,6055765.0,691863980.0,0.0,0.0,0.0,3.696944e+09,A1CAP,2023
4,8.297508e+09,9773361.0,31826374.0,4.065838e+09,0.0,28062814.0,0.0,0.0,0.0,144667639.0,...,0.0,160295553.0,5169826.0,369461172.0,0.0,0.0,0.0,4.065838e+09,A1CAP,2024


In [17]:
# 2. Usuń 'indeks', jeśli istnieje
df = df.drop(columns=['indeks', 'Źródło_pliku'], errors='ignore')

# 3. Identyfikator i rok
id_col = 'Nazwa'
rok_col = 'Rok'

# 4. Przekształcenie: iterujemy po wszystkich kolumnach oprócz 'Nazwa' i 'Rok'
value_cols = [col for col in df.columns if col not in [id_col, rok_col]]

# 5. Tworzymy nowy DataFrame z odpowiednio nazwanymi kolumnami
dfs = []
for col in value_cols:
    temp = df[[id_col, rok_col, col]].copy()
    temp.columns = [id_col, rok_col, 'wartość']
    temp['kolumna'] = f"{col}"
    temp['nazwa_cechy_rok'] = temp['kolumna'] + '_' + temp[rok_col].astype(str)
    dfs.append(temp[[id_col, 'nazwa_cechy_rok', 'wartość']])

# 6. Scal wszystkie fragmenty
df_long = pd.concat(dfs, axis=0)

# 7. Pivot do formatu szerokiego
df_wide = df_long.pivot_table(
    index=id_col,
    columns='nazwa_cechy_rok',
    values='wartość',
    aggfunc='first'
).reset_index()

# 8. Spłaszcz kolumny
df_wide.columns.name = None

# 9. Podgląd
df_wide.head()

,Nazwa,(Głównie poza działalnością) Koszty finansowe (-)_2008,(Głównie poza działalnością) Koszty finansowe (-)_2009,(Głównie poza działalnością) Koszty finansowe (-)_2010,(Głównie poza działalnością) Koszty finansowe (-)_2011,(Głównie poza działalnością) Koszty finansowe (-)_2012,(Głównie poza działalnością) Koszty finansowe (-)_2013,(Głównie poza działalnością) Koszty finansowe (-)_2014,(Głównie poza działalnością) Koszty finansowe (-)_2015,(Głównie poza działalnością) Koszty finansowe (-)_2016,...,Żywe istoty_2015,Żywe istoty_2016,Żywe istoty_2017,Żywe istoty_2018,Żywe istoty_2019,Żywe istoty_2020,Żywe istoty_2021,Żywe istoty_2022,Żywe istoty_2023,Żywe istoty_2024
0,A1CAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1,ACSEL,NaN,NaN,-146658.0,0.0,-186001.0,-261542.0,-261542.0,-299978.0,-30594.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADEL,-4467984.0,-4467984.0,-2562968.0,-2482437.0,-4188750.0,-5540038.0,-5540038.0,-28096595.0,-25598000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADESE,NaN,NaN,-10952999.0,-12097631.0,-22325339.0,-8613300.0,-8613300.0,-20467422.0,-34318313.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADGYO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [22]:
from sklearn.model_selection import train_test_split



# podział na zbiór treningowy walidacyjni i testowy
train_df, temp_df = train_test_split(df_wide, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Zapisywanie do plików
train_df.to_csv("Data/Concatenated Features/raw_features_train.csv", index=False, encoding='utf-8-sig')
val_df.to_csv("Data/Concatenated Features/raw_features_val.csv", index=False, encoding='utf-8-sig')
test_df.to_csv("Data/Concatenated Features/raw_features_test.csv", index=False, encoding='utf-8-sig')